In [2]:
# Dependencies
import requests
import json
from configET import nyt_api_key
from pprint import pprint
import pandas as pd
from datetime import date

In [3]:
# # ****DO NOT RUN THIS AGAIN***** 
# #Request articles from NYT Most Popular API from 1,7, and 30 days ago 
# #LAST RUN ON 4/13/2020 AKA '041320'
# today = date.today()
# date = today.strftime("%m%d%y")

# periods = [1,7,30]

# #loop through periods

# for period in periods:
#     #set urls
#     most_emailed_url = f"https://api.nytimes.com/svc/mostpopular/v2/emailed/{period}.json?"
#     most_viewed_url = f"https://api.nytimes.com/svc/mostpopular/v2/viewed/{period}.json?"
#     most_shared_url = f"https://api.nytimes.com/svc/mostpopular/v2/shared/{period}.json?"
#     most_urls = [most_emailed_url,most_viewed_url,most_shared_url]
    
#     #loop through urls and store responses
    
#     most_responses = []
#     for u in most_urls:
#         query_url = u + "api-key=" + nyt_api_key
#         # Request articles
#         most_responses.append(requests.get(query_url).json())
    
#     #as of 4/13/2020 (3/14/2020-4/13/2020)
#     #pull results from list
#     most_emailed = most_responses[0]
#     most_viewed = most_responses[1]
#     most_shared = most_responses[2]
    
#     #create dataframes
#     most_emailed_df = pd.DataFrame(most_emailed['results'])
#     most_viewed_df = pd.DataFrame(most_viewed['results'])
#     most_shared_df = pd.DataFrame(most_shared['results'])

#     #save as CSVs
#     most_emailed_df.to_csv(f'most_emailed_{period}_{date}.csv') # monthdayyear
#     most_viewed_df.to_csv(f'most_viewed_{period}_{date}.csv') # monthdayyear
#     most_shared_df.to_csv(f'most_shared_{period}_{date}.csv') # monthdayyear


In [4]:
#info we want: asset_id keywords, title, published_date, url, resave partial data frame to csv
period = 30
date = '041320' # '041320'
#read CSVs and create partial data frame with relevant info
most_emailed_df = pd.read_csv(f'data/full_csv/most_emailed_{period}_{date}.csv')
most_emailed_df = most_emailed_df[['title','asset_id','adx_keywords','published_date','url']]


most_viewed_df = pd.read_csv(f'data/full_csv/most_viewed_{period}_{date}.csv')
most_viewed_df = most_viewed_df[['title','asset_id','adx_keywords','published_date','url']]

most_shared_df = pd.read_csv(f'data/full_csv/most_shared_{period}_{date}.csv')
most_shared_df = most_shared_df[['title','asset_id','adx_keywords','published_date','url']]


In [8]:
#data frames: 
# "data/most_emailed_partial_30_041320.csv"
# "data/most_viewed_partial_30_041320.csv"
#" data/most_shared_partial_30_041320.csv"

dataframes = [most_emailed_df,most_viewed_df,most_shared_df]
for data in dataframes:
    data.sort_values(by="published_date",ascending = True, inplace=True)
    #make keywords a list
    first_keyword = []
    contains_covid19 = []
    for index,row in data.iterrows():
        key_list = row['adx_keywords'].split(';')
        if key_list[0] == 'Coronavirus (2019-nCoV)':
            first_keyword.append('True')
        else:
            first_keyword.append('False')
       
        if 'Coronavirus (2019-nCoV)' in key_list:
            contains_covid19.append('True')
        else:
            contains_covid19.append('False')
    data['first_keyword'] = first_keyword
    data['about_covid19'] = contains_covid19
    data.set_index('asset_id',inplace=True)
    
#save partial dataframe as CSV
most_emailed_df.to_csv(f'data/most_emailed_partial_{period}_{date}.csv') # monthdayyear 
most_viewed_df.to_csv(f'data/most_viewed_partial_{period}_{date}.csv') # monthdayyear
most_shared_df.to_csv(f'data/most_shared_partial_{period}_{date}.csv') # monthdayyear

In [19]:
publish_count_df = []
for data in dataframes:
    count_df = pd.DataFrame(data['published_date'].value_counts().reset_index().values,
                            columns=['Published Date','Number Articles'])
    count_df.sort_values(by="Published Date",ascending = True, inplace=True)
    count_df.set_index('Published Date',inplace=True)
    publish_count_df.append(count_df)

most_emailed_pub_df = publish_count_df[0]
most_viewed_pub_df = publish_count_df[1] 
most_shared_pub_df = publish_count_df[2]

most_emailed_pub_df.to_csv(f'data/most_emailed_pubcount_{period}_{date}.csv')
most_viewed_pub_df.to_csv(f'data/most_viewed_pubcount_{period}_{date}.csv')
most_shared_pub_df.to_csv(f'data/most_shared_pubcount_{period}_{date}.csv')

In [20]:
most_emailed_pub_df.head()

,Number Articles
Published Date,
2020-03-12,1
2020-03-15,1
2020-03-17,1
2020-03-19,3
2020-03-20,1


In [6]:
# #Top Engagement:
# most_engaged_df = pd.merge(most_emailed_df,most_viewed_df, on='asset_id', how='inner')
# most_engaged_df = pd.merge(most_engaged_df,most_shared_df, on='asset_id', how='inner')
# most_engaged_df.head()

In [101]:
# # Testing NYT Article Search API
# search_url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"
# query = "coronavirus"
# start = "20191231"
# end = "20200409"

# # Build query URL
# query_url = search_url + "api-key=" + nyt_api_key + "&q=" + query + "begin_date=" + start + "end_date=" + end

# # Request articles
# articles = requests.get(query_url).json()

# # The "response" property in articles contains the actual articles
# # list comprehension.
# articles_list = articles["response"]["docs"]


In [16]:
# Testing NTY archive API
# year = 2020
# month = 3
# archive_url = f"https://api.nytimes.com/svc/archive/v1/{year}/{month}.json?"
# query_url = archive_url + "api-key=" + nyt_api_key
# march_articles = requests.get(query_url).json()